In [387]:
#Подключение небходимых библиотек
import qiskit, qiskit_aer
import qiskit.visualization
import qiskit_ibm_runtime
import random
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [388]:
#service = qiskit_ibm_runtime.QiskitRuntimeService(channel ='ibm_quantum', token='17efa62aa9873b5a041acbd771297e673778205cc443bc748e869e84330c331fc92ce25cd9f5b87f1432106dd7983eedac0deedc237296b7ed815d6200f31b5a')

In [389]:
qubits_num = 3 #количество необходимых кубитов
secret_length = 10 #длина секрета
secret = []
for i in range(secret_length): #генерация случайного секрета
    bit = random.randint(0,1)
    secret.append(bit)

In [390]:
#функция генерации ghz состояния
def generate_ghz():
    qc = qiskit.QuantumCircuit(qubits_num, qubits_num) #указываем количество 
    #кубитов и классических регистров
    
    qc.h(0) #применяем гейт Адамара
    qc.cx(0,1) # и гейты CNOT
    qc.cx(0,2)
    return qc

In [391]:
#функция кодирования секрета
def secret_encode(secret):
    schemes = [] #массив ghz состояний, содержащих весь секрет
    for bit in secret:
        qc = generate_ghz() #генерируем ghz состояние
        if bit == 1: #если кодируем бит "1", то применяем гейт Паули Z, 
            #иначе ничего не делаем
            
            qc.z(0)
        schemes.append(qc)
    return schemes

In [392]:
ts1 = time.time()
#производим кодирование секрета
encoded_secret = secret_encode(secret)

In [393]:
#функция выбора базисов
def generate_bases():
    bases = [] #выбранные базисы
    y_bases = 1
    while y_bases % 2 != 0: #случайно выбираем базисы до тех пор пока Y 
        #базисов не будет четно
        bases = []
        for i in range(3):
            rand = random.randint(0,1)
            if rand == 0:
                bases.append('x')
            else:
                bases.append('y')
        y_bases = bases.count('y')
    return bases

In [394]:
#функция проведения измерений в базисах X и Y
def bases_measurment(scheme, bases):
    for i in range(3):
        if bases[i] == 'x':
            scheme.h(i) #применяется гейт Адамара
        else:
            scheme.sdg(i) #применяется гейт sdg, затем гейт Адамара
            scheme.h(i)
        scheme.measure(i,i) #измерение i-ого кубита и запись результата в 
            #i-й классический регистр
    return scheme

In [395]:
#функция восстановления бита секрета по известной информации
def bit_recovering(bases, measures):
    if bases[1] == 'x' and bases[2] == 'x':
        if (measures[1] == 0 and measures[2] == 0) or (measures[1] == 1 and 
                                                       measures[2] == 1):
            if measures[0] == 0:
                return 0
            else:
                return 1
        elif (measures[1] == 0 and measures[2] == 1) or (measures[1] == 1 and 
                                                         measures[2] == 0):
            if measures[0] == 1:
                return 0
            else:
                return 1
                
    elif bases[1] == 'y' and bases[2] == 'y':
        if (measures[1] == 0 and measures[2] == 0) or (measures[1] == 1 and  
                                                       measures[2] == 1):
            if measures[0] == 1:
                return 0
            else:
                return 1
        elif (measures[1] == 0 and measures[2] == 1) or (measures[1] == 1 and  
                                                         measures[2] == 0):
            if measures[0] == 0:
                return 0
            else:
                return 1
                
    elif (bases[1] == 'y' and bases[2] == 'x') or (bases[1] == 'x' and 
                                                   bases[2] == 'y'):
        if (measures[1] == 0 and measures[2] == 0) or (measures[1] == 1 and 
                                                       measures[2] == 1):
            if measures[0] == 1:
                return 0
            else:
                return 1
        elif (measures[1] == 0 and measures[2] == 1) or (measures[1] == 1 and  
                                                         measures[2] == 0):
            if measures[0] == 0:
                return 0
            else:
                return 1

In [396]:
#функция восстановления секрета
def secret_decode(schemes):
    recovered_secret = [] #восстанавливаемый секрет
    for scheme in schemes:
        measures = []
        bases = generate_bases() #генерируем базисы

        #проведение измерений
        scheme = bases_measurment(scheme, bases) #проводим измерения
        
        backend = qiskit_aer.AerSimulator() #подключение симулятора
        
        circuit = qiskit.transpile(scheme, backend) #транспиляция схемы

        job = backend.run(circuit, shots=1) #запускаем схему по одному разу для 
        #каждого бита
        
        result = job.result() #считываем результаты измерений
        counts=result.get_counts()

        measures = list(map(int, list(counts.keys())[0])) #преобразуем полученные 

        #результаты в массив битов
        
        recovered_bit = bit_recovering(bases, measures) #восстанавливаем отдельный 
        #бит
        recovered_secret.append(recovered_bit) #добавляем бит к восстанавливаемому 
        #секрету

    return recovered_secret
            
        

In [397]:
recovered_secret = secret_decode(encoded_secret) #применяем функцию декодирования 
#секрета
ts2 = time.time()
errors = 0 #количество ошибок
for i in range(secret_length): #подсчитываем количество ошибок
    if secret[i] != recovered_secret[i]:
        errors += 1
print(errors)
print(ts2-ts1)

0
0.4920797348022461


In [398]:
total_qubits_num = 127 #количество кубит квантового процессора
qubits_num = 3 #количество необходимых кубитов для передачи одного бита секрета
secret_length = total_qubits_num//3 #длина секрета
secret = []
for i in range(secret_length): #генерация случайного секрета
    bit = random.randint(0,1)
    secret.append(bit)

In [399]:
#функция генерации ghz состояний на квантовом процессоре
def generate_ghz_real():
    qc = qiskit.QuantumCircuit(total_qubits_num, total_qubits_num) #указываем
    #количество кубитов и классических регистров
    for i in range(0, total_qubits_num - qubits_num + 1, qubits_num):
        qc.h(i) #применяем гейт Адамара
        qc.cx(i,i+1) # и гейты CNOT
        qc.cx(i,i+2)
    return qc

In [400]:
#функция кодирования секрета для квантового процессора
def secret_encode_real(secret):
    qc = generate_ghz_real() #генерируем ghz состояние
    for i in range(len(secret)):
        if secret[i] == 1: #если кодируем бит "1", то применяем гейт Паули Z, 
            #иначе ничего не делаем
            qc.z(i*3)
    return qc

In [401]:
#производим кодирование секрета
encoded_secret_real = secret_encode_real(secret)

In [402]:
#функция проведения измерений в базисах X и Y для квантового процессора
def bases_measurment_real(scheme, bases):
    for i in range(total_qubits_num//3): #измерение трех кубитов за итерацию
        for j in range(3):
            if bases[i][j] == 'x':
                scheme.h(i*3+j) #применяется гейт Адамара
            else:
                scheme.sdg(i*3+j) #применяется гейт sdg, затем гейт Адамара
                scheme.h(i*3+j)
            scheme.measure(i*3+j,i*3+j) #измерение кубита и запись  
            #результата в соответстующий ему классический регистр
    return scheme

In [403]:
bases_output = []
measurment_output = []
#функция восстановления секрета на квантовом процессоре
def secret_decode_real(scheme):
    recovered_secret = [] #восстанавливаемый секрет
    measures = []
    bases = []
    for i in range(total_qubits_num//3):
        bases.append(generate_bases()) #генерируем тройки базисов

    for i in bases:
        bases_output.append(i)

    scheme = bases_measurment_real(scheme, bases) #проводим измерения
    backend = qiskit_aer.AerSimulator()
    #backend = service.least_busy(operational=True, simulator=False) #вместо симулятора выбираем  
    #квантовое устройство, в данном случае ibm_brisbane

    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    circuit = pm.run(scheme)
    sampler = qiskit_ibm_runtime.Sampler(mode=backend, options={"default_shots": 1})

    job = sampler.run([circuit]) #запускаем схему на квантовом процессоре 
    #один раз

    result = job.result() #считываем результаты измерений
    counts = result[0].data.c.get_counts()
    count = list(counts.keys())[0]
    for i in count:
        measures.append(int(i))
    #преобразуем полученные результаты в массив троек
    measures = measures [(total_qubits_num-secret_length*3):]
    measures.reverse() 

    for i in measures:
        measurment_output.append(i)

    #восстанавливаем биты секрета
    for i in range(len(measures)//3):
        
        triplet_measure = measures[i*3:(i+1)*3] #тройка результатов измерений

        recovered_bit = bit_recovering(bases[i], triplet_measure)

        recovered_secret.append(recovered_bit) #добавляем бит к восстанавливаемому 
        #секрету

    return recovered_secret


In [404]:
recovered_secret = secret_decode_real(encoded_secret_real) #применяем функцию 
#декодирования секрета

errors = 0 #количество ошибок
for i in range(secret_length): #подсчитываем количество ошибок
    if secret[i] != recovered_secret[i]:
        errors += 1

qber = errors/secret_length*100 #QBER - величина, показывает процент ошибок
print("QBER =", qber,"%")

QBER = 16.666666666666664 %


In [407]:

str1 = ""
for i in bases_output:
    str2 = ""
    for j in i:
        str2 += str(j)
    str1 += str2
    str1 += " "
print(str1)

str1 = ""
for i in range(0,len(measurment_output),3):
    str1+=str(measurment_output[i])
    str1+=str(measurment_output[i+1])
    str1+=str(measurment_output[i+2])
    str1+=" "
print(str1)

str1 = ""
for i in recovered_secret:
    str1+=str(i)

print(str1)

str1 = ""
for i in secret:
    str1+=str(i)

print(str1)

xyy yyx xxx yxy xxx yxy yxy yyx xxx yyx xxx xxx yxy yyx yxy yxy xyy xyy yxy xxx yyx xxx yyx yyx yxy xxx xxx yyx xyy xyy yyx xyy yxy yxy xxx yyx yyx xxx yxy xxx yyx xxx 
000 110 001 010 000 111 011 000 011 001 000 101 011 011 101 011 000 100 011 010 001 001 010 110 110 110 001 111 000 011 110 110 010 111 100 001 111 111 111 100 110 011 
111000110000111110110101101011110010010110
111010111000001010110101100111110010010110


In [406]:
#qiskit.visualization.plot_histogram(counts)
#backend = service.backend(name="ibm_brisbane")
#scheme.draw(output='mpl')